In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 15:03:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import math
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('drive/MyDrive/dacon/data_re/상추/train_log_1203.csv').iloc[:,1:]
test = pd.read_csv('drive/MyDrive/dacon/data_re/상추/train_log_1203.csv').iloc[:,1:]

In [6]:
corr_df = train.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
s = corr_df.unstack()
성장 = pd.DataFrame(s['predicted_weight_g']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
성장.sort_values(by=[0],ascending=False).reset_index().to_csv('target값이랑corr.csv',index=False,encoding='utf-8-sig')
under_1per = 성장[성장[0]<0.1].index
under_1per

Index(['시간당총광량9~11', '내부온도관측치19~21', '시간당백색광량20~22', '내부온도관측치2~4',
       '시간당백색광량21~23', 'day_red_weight_std', '내부온도관측치6~8', '내부온도관측치20~22',
       '시간당총광량8~10', 'day_water_weight_std', '시간당적색광량16~18',
       'day_blue_weight_std', '내부온도관측치21~23', '시간당적색광량14~16', '시간당적색광량15~17',
       '시간당적색광량13~15', '시간당백색광량0~2', '시간당적색광량17~19', '내부온도관측치5~7',
       '내부온도관측치3~5', '일간누적백색광량2', '일간누적백색광량5', '시간당총광량7~9', '시간당적색광량12~14',
       '시간당적색광량18~20', '내부온도관측치4~6', '시간당청색광량12~14', '시간당청색광량11~13',
       '시간당청색광량10~12', '시간당총광량6~8', '시간당분무량1~3', '시간당적색광량11~13', '시간당청색광량9~11',
       '시간당청색광량8~10', '시간당청색광량13~15', '시간당적색광량19~21', '시간당청색광량7~9',
       '시간당적색광량10~12', '시간당백색광량1~3', '시간당백색광량2~4', '시간당백색광량3~5'],
      dtype='object')

In [7]:
drop_column = train.copy()
drop_column = drop_column.drop(under_1per,axis=1)

In [8]:
!pip install lightgbm==3.3.2
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (25.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program i

In [9]:
import lightgbm as lgb

In [10]:
lgb.__version__

'3.3.2'

In [11]:
X = drop_column.drop(['predicted_weight_g'],axis=1)
y = drop_column['predicted_weight_g']

In [25]:
SEED = 404
params = {
        'boosting': 'gbdt',
        'objective': 'regression', 
        # "verbose" : 0,
        # "num_leaves" : 63,
         "learning_rate" : 0.05,
          'metric': 'rmse',
        # "feature_fraction" : 1.0,
        # "max_bin": 511,
        # 'min_data_in_leaf': 20,
        # 'bagging_seed': SEED,
        # 'feature_fraction_seed': SEED,
        # 'data_random_seed': SEED,
        # 'objective_seed': SEED,
        # 'drop_seed': SEED,
        'seed': SEED
    }
lgtrain = lgb.Dataset(X, label = y)
model = lgb.train( params, lgtrain ,1000, verbose_eval=100)  

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29514
[LightGBM] [Info] Number of data points in the train set: 784, number of used features: 208
[LightGBM] [Info] Start training from score 34.919681
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [13]:
drop_columntt = test.copy()
drop_columntt = drop_columntt.drop(under_1per,axis=1)

In [26]:
testX = drop_columntt.drop(['predicted_weight_g'],axis=1)
pred_Y = model.predict(testX) 

In [27]:
pred_Y

array([1.64867544e-01, 1.87847968e-01, 2.75620566e-01, 4.02936302e-01,
       4.85294936e-01, 1.93625973e+00, 2.32361959e+00, 3.49639906e+00,
       4.45621594e+00, 5.33376964e+00, 6.23232796e+00, 7.77686919e+00,
       9.44688744e+00, 1.12173505e+01, 1.43422242e+01, 1.77825412e+01,
       2.16891740e+01, 2.65742405e+01, 3.24984350e+01, 3.95533325e+01,
       4.75261525e+01, 5.60070320e+01, 6.54939609e+01, 7.45600935e+01,
       8.09655947e+01, 8.29926436e+01, 8.25650837e+01, 8.20738581e+01,
       1.04428614e+00, 1.22210893e+00, 1.33411829e+00, 1.45562478e+00,
       1.60160198e+00, 1.81383074e+00, 2.28372026e+00, 2.81712856e+00,
       3.40256770e+00, 4.17209536e+00, 4.99100729e+00, 5.80907718e+00,
       7.10063517e+00, 8.70117824e+00, 1.02212859e+01, 1.24613438e+01,
       1.48236409e+01, 1.79614590e+01, 2.16680372e+01, 2.69008770e+01,
       3.11771190e+01, 3.30617578e+01, 4.15986804e+01, 4.88464247e+01,
       5.28044125e+01, 5.53183874e+01, 5.68536771e+01, 6.53091927e+01,
      